In [1]:
import Pkg; 

cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol
using PyCall
using JLD2
using LinearAlgebra
using NearestNeighbors
using Rotations
using GLMakie

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [2]:
function get_boundary_filtration(points, n_atoms_per_mol)
    py"""
    import oineus as oin
    import numpy as np
    import torch
    import diode

    def get_boundary_filtration(points, n_atoms_per_mol):
        points = np.asarray(points)

        simplices = diode.fill_alpha_shapes(points)

        fil = oin.Filtration_double([oin.Simplex_double(s[0], s[1]) for s in simplices])
        
        def is_multi(sigma):
            has_a = has_b = False
            for v in sigma.vertices:
                if v < n_atoms_per_mol:
                    has_a = True
                else:
                    has_b = True
            return has_a and has_b
        
        fil = fil.subfiltration(is_multi)

        return fil
    """
    py"get_boundary_filtration"(points, n_atoms_per_mol)
end

get_boundary_filtration (generic function with 1 method)

In [4]:
@load "assets/output/persistence/exploding.jld2" input output

#template_centers = MorphoMol.Utilities.TMV_TEMPLATES["6r7m"]["template_centers"]
#x_init = MorphoMol.Utilities.get_initial_state(2, 150.0)

points = MorphoMol.Utilities.get_matrix_realization(output["states"][1], input["template_centers"])
points = [e for e in eachcol(hcat(points...))]

2412-element Vector{SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}}:
 [-9.566561876644581, 17.509013521627477, 71.58452821235196]
 [-8.500446789019989, 18.01219303064911, 72.4387980769966]
 [-8.117724535767472, 16.991918043372966, 73.50076635675074]
 [-8.861590174543984, 16.051673381858517, 73.76502737911528]
 [-8.916337796509154, 19.321046228887923, 73.10918526015989]
 [-9.931535276755326, 19.1002323266637, 74.07121659888348]
 [-6.946220074127353, 17.17742496304515, 74.09599528675251]
 [-6.563609898130871, 16.44640419564484, 75.29327056456184]
 [-7.006586875335863, 17.22935323611511, 76.52195083122727]
 [-7.012883297798869, 18.462167161856573, 76.52152708353502]
 [-5.051001646099358, 16.229956943262216, 75.35413814889166]
 [-4.491026029748287, 15.20513546970669, 74.39831690177914]
 [-4.824191368679596, 13.863371574280334, 74.51285294791784]
 ⋮
 [63.041204249597754, -14.172555315381889, 30.14021829952031]
 [63.13743579076346, -13.22457312449928

In [64]:
function get_multichromatic_edges_of_tetrahedron(verts; split = 1206)
    a,b,c,d = verts
    combinations = [[a,b], [a,c], [a,d], [b,c], [b,d], [c,d]]
    [e for e in combinations if any([v > split for v in e]) && any([v <= split for v in e])]
end

function get_barycenters_of_multichromatic_edges(edges, points)
    barycenters = Vector{Point3f}([])
    for e in edges
        a,b = e[1],e[2]
        bc = Point3f(0.5 * (points[a] + points[b]))
        push!(barycenters, bc)
    end
    if length(barycenters) == 4
        bc = Point3f(0.25 * (barycenters[1] + barycenters[2] + barycenters[3] + barycenters[4]))
        push!(barycenters, bc)
    end 
    barycenters
end

function get_faces_from_barycenters(barycenters)
    if length(barycenters) == 3
        faces = [1 2 3]
    elseif length(barycenters) == 5
        faces = [1 2 5; 3 4 5; 2 4 5; 1 3 5]
        #3 4 5; 1 4 5
    end
    faces
end

get_faces_from_barycenters (generic function with 1 method)

In [65]:
f = Figure(fontsize = 7)
mesh_ax = Axis3(f[1,1])
tetrahedron = [Point3f(0.0, 0.0, 0.0), Point3f(2.0, 0.0, 0.0), Point3f(1.0, 0.0, 2.0), Point3f(1.0, 2.0, 1.0)]


mce = get_multichromatic_edges_of_tetrahedron([1,2,3,4], split = 2)
bcs = get_barycenters_of_multichromatic_edges(mce, tetrahedron)
faces = get_faces_from_barycenters(bcs)

mesh!(bcs, faces)


display(f)

GLMakie.Screen(...)

In [69]:
f = Figure(fontsize = 7)
mesh_ax = Axis3(f[1,1])

for c in fil.cells()
    if length(c.vertices) == 4
        mce = get_multichromatic_edges_of_tetrahedron(c.vertices)
        bcs = get_barycenters_of_multichromatic_edges(mce, points)
        faces = get_faces_from_barycenters(bcs)

        mesh!(bcs, faces)
    end
end

display(f)

GLMakie.Screen(...)

In [14]:
f = Figure(fontsize = 7)
conf_ax = Axis3(f[1,1])

c1_points = [Point3f(p) for p in points[1:1206]]
c2_points = [Point3f(p) for p in points[1207:end]]

conf_ms = 10
scatter!(conf_ax, c1_points, markersize = conf_ms)
scatter!(conf_ax, c2_points, markersize = conf_ms)

for c in fil.cells()
    if length(c.vertices) == 3
        i = c.vertices[1]
        j = c.vertices[2]
        k = c.vertices[3]
        a = Point3f(points[i])
        b = Point3f(points[j])
        c = Point3f(points[k])
        c1 = i <= 1206 ? 1 : 2
        c2 = j <= 1206 ? 1 : 2
        c3 = k <= 1206 ? 1 : 2

        lines!(conf_ax, [a,b], color = [c1, c2])
        lines!(conf_ax, [a,c], color = [c1, c3])
        lines!(conf_ax, [a,b], color = [c2, c3])
    end
end

mesh_ax = Axis3(f[1,2])

for c in fil.cells()
    if length(c.vertices) == 4
        h = c.vertices[1]
        i = c.vertices[2]
        j = c.vertices[3]
        k = c.vertices[4]

        mce = get_multichromatic_edges_of_tetrahedron(c.vertices)
        bcs = get_barycenters_of_multichromatic_edges(mce, points)
        faces = get_faces_from_barycenters(bcs)

        mesh!(bcs, faces)
    end
end

display(f)

GLMakie.Screen(...)